# Object-oriented context-free grammar

Objects exist in superposition
When property (object) is referenced, return random from the set and collapse the reference.

```
Fruit:{
    {color}
    {pit}
}

Apple:{
    {color}: <Red, Green>
    name: 'Apple'
}

Pear:{
    {color}: <Red, Green, Yellow>
    name: 'Pear'
}
```

```
f = <Fruit>
update:
    <f> = <Apple, Pear>
    <f.color> = Union(f.{color} for f in {f}) = <Red, Green, Yellow>
```


Now if we reference `Collapse(f)`, it will first check if f is collapsed. if yes, return concrete value. if no, collapse it.

Nested referencing: `color = Collapse(f.color)`: 
We return `color = Collapse(<f.color>)`
then, if f is not collapsed, we restrict to f.color ⊃ color

```
f = <Apple, Pear>

Collapse(f.color):
f.color = <Red, Green, Yellow>

case 1: color = Red
    <f>.color = Red
    
    restrict:
    get all parents f, f.Restrict(color=Red)
        <f> = <Apple, Pear>
        
    update:
        <f>.color = Red
        <f>.name = <'Apple', 'Pear'>

case 2: color = Yellow
    <f>.color = Yellow
    
    restrict:
    get all parents f, f.Restrict(color=Yellow)
        <f> = <Pear>
        f = Pear
        
    update:
        <f>.color = Yellow
        <f>.name = <'Pear'>
```

So objects have a notion of parent. anything that is a property of an object has its parent
set to be the object.

```
object.Restrict(property, value):
    new_set = set()
    for object in set:
        if object.property ⊃ value:
            new_set.add(object)
    set = new_set
    for parent, property in parents:
        parent.restrict(property, this)
```


# Implementation

1. Object with some magic in its constructor
2. Normal object, with decorators on its properties

In [622]:
import random
import inspect
import traceback


In [875]:
class Symbol(object):
    def __init__(self, values):
        self.InitVals(values)
        
    def InitVals(self, values):
        # TODO: what if value is singleton
        if inspect.isclass(values):
            self.values = []
            for subclass in values.__subclasses__():
                print(subclass)
                x = subclass()
                self.values.append(x)
                for key in x.__dict__:
                    print(key)
                for t in subclass.mro():
                    for key in t.__dict__:
                        if key[:2] != '__':
                            print(key)
#                     print(t.__dict__)
        else:
            self.values = values
#             print(values)
        self.value = None
        
    def Ref(self, instance=None, prop_name=None):
        if self.value:
            return self.value
        else:
            if prop_name is not None:
                print(prop_name + ' collapsing')
                print(instance)
                stack = traceback.extract_stack()
                lines = traceback.format_list(stack)
                for line, stick in zip(lines, stack):
                    if prop_name in line:
                        refs = line.split('\n')[-2].strip().split('.')
                if prop_name is not refs[-1]:
                    return self
            return self.Collapse()
        
    def Collapse(self):
        if self.value:
            return self.value
        self.value = random.choice(self.values)
        return self.value
    
    def Update(self):
        pass

class Symbolic(object):      
    def __init__(self):
        self.symbols = WeakKeyDictionary()
        
    def __get__(self, instance_obj, objtype):
        if instance_obj == None:
            return self
        prop_name = None
        for t in objtype.mro():
            for key, val in t.__dict__.items():
                if id(val) == id(self):
                    prop_name = key
        return self.symbols[instance_obj].Ref(instance=instance_obj, prop_name=prop_name)
    
    def __set__(self, instance, val):
        s = Symbol(val)
        s.symbolic = self
        self.symbols[instance] = s
        
    def __delete__(self, instance):
        del self.age[instance]

class Fruit(object):
    color = Symbolic()
    def __init__(self, name):
        self.name = name

class Apple(Fruit):
    def __init__(self):
        self.name = 'Apple'
        self.color = ['red', 'green']
        
class Pear(Fruit):
    def __init__(self):
        self.name = 'Pear'
        self.color = ['red', 'yellow']

        
        
class State(object):
    fruit = Symbolic()
    def __init__(self):
        self.fruit = Fruit

In [876]:
state = State()

<class '__main__.Apple'>
name
color
<class '__main__.Pear'>
name
color


In [823]:
state.fruit.color

fruit collapsing


AttributeError: 'Symbol' object has no attribute 'color'

1. collapse sub attribute but not main
3. be able to pass references without collapse
4. inspect symbol values / etc without collapse

In [811]:
list(State.fruit.symbols.values())[0].values

[<__main__.Apple at 0x104dc0748>, <__main__.Pear at 0x104dc0240>]

Add to symbol's dictionary to build out its values